In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
# import scipy
# from netcal.metrics import ECE, ACE

from dataloader import get_dataloaders
from models.msdnet_ge import MSDNet
from models.msdnet_imta import IMTA_MSDNet
from utils import parse_args
from utils_notebook import f_probs_ovr_poe_logits_weighted, modal_probs_decreasing, f_probs_ovr_poe_logits_weighted_generalized, \
            anytime_caching, get_metrics_for_paper, get_metrics_with_error_bars, get_logits_targets_imta_image_net

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from collections import OrderedDict, Counter
import random
import os
from typing import Dict

from tqdm import tqdm

/home/metod/miniconda3/envs/imta/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
logits, targets = get_logits_targets_imta_image_net(model_name='model1_IMTA', epoch='59', model_name_pretrained='model1', epoch_pretrained='89', dataset='ImageNet')

building network of steps: 
[4, 4, 4, 4, 4] 20
 ********************** Block 1  **********************
|		inScales 4 outScales 4 inChannels 32 outChannels 16		|

|		inScales 4 outScales 4 inChannels 48 outChannels 16		|

|		inScales 4 outScales 4 inChannels 64 outChannels 16		|

|		inScales 4 outScales 4 inChannels 80 outChannels 16		|

 ********************** Block 2  **********************
|		inScales 4 outScales 4 inChannels 96 outChannels 16		|

|		inScales 4 outScales 3 inChannels 112 outChannels 16		|
|		Transition layer inserted! (max), inChannels 128, outChannels 64	|

|		inScales 3 outScales 3 inChannels 64 outChannels 16		|

|		inScales 3 outScales 3 inChannels 80 outChannels 16		|

 ********************** Block 3  **********************
|		inScales 3 outScales 3 inChannels 96 outChannels 16		|

|		inScales 3 outScales 3 inChannels 112 outChannels 16		|

|		inScales 3 outScales 2 inChannels 128 outChannels 16		|
|		Transition layer inserted! (max), inChannels 144, outChannels

In [6]:
logits.shape

torch.Size([5, 50000, 1000])

In [7]:
acc, mono_modal, mono_correct = get_metrics_for_paper(logits, targets, model_name='IMTA')

In [12]:
mono_modal

{'IMTA': [79.162, 61.17, 42.818, 30.452, 15.556, 10.884, 5.71, 1.184],
 'IMTA-PA': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'IMTA-CA': [4.704, 4.006, 2.592, 1.588, 0.594, 0.31, 0.122, 0.01]}

In [13]:
mono_correct

{'IMTA': [82.24, 60.814, 40.944, 29.096, 15.462, 11.318, 6.688, 1.864],
 'IMTA-PA': [1.746, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'IMTA-CA': [19.558, 14.512, 9.326, 6.332, 3.08, 2.198, 1.252, 0.332]}

In [4]:
MODEL = 'IMTA'
DATASET = 'ImageNet'
MODELS = [('model1_IMTA', 59, 'model1', 89), ('model_seed_1_IMTA/save_models', 89, 'models_ge_seed_1/save_models', 89), 
            ('model_seed_2_IMTA/save_models', 89, 'models_ge_seed_2/save_models', 89)]

In [5]:
get_metrics_with_error_bars(model_name=MODEL, dataset=DATASET, model_list=MODELS)

building network of steps: 
[4, 4, 4, 4, 4] 20
 ********************** Block 1  **********************
|		inScales 4 outScales 4 inChannels 32 outChannels 16		|

|		inScales 4 outScales 4 inChannels 48 outChannels 16		|

|		inScales 4 outScales 4 inChannels 64 outChannels 16		|

|		inScales 4 outScales 4 inChannels 80 outChannels 16		|

 ********************** Block 2  **********************
|		inScales 4 outScales 4 inChannels 96 outChannels 16		|

|		inScales 4 outScales 3 inChannels 112 outChannels 16		|
|		Transition layer inserted! (max), inChannels 128, outChannels 64	|

|		inScales 3 outScales 3 inChannels 64 outChannels 16		|

|		inScales 3 outScales 3 inChannels 80 outChannels 16		|

 ********************** Block 3  **********************
|		inScales 3 outScales 3 inChannels 96 outChannels 16		|

|		inScales 3 outScales 3 inChannels 112 outChannels 16		|

|		inScales 3 outScales 2 inChannels 128 outChannels 16		|
|		Transition layer inserted! (max), inChannels 144, outChannels

({'IMTA': ([0.5774, 0.6627, 0.7033, 0.7216, 0.729],
   [0.0045, 0.0036, 0.0025, 0.0025, 0.0024]),
  'IMTA-PA': ([0.5774, 0.6563, 0.6965, 0.7173, 0.7319],
   [0.0045, 0.0018, 0.0025, 0.0026, 0.0015]),
  'IMTA-CA': ([0.5774, 0.6512, 0.6905, 0.711, 0.7238],
   [0.0045, 0.0036, 0.0026, 0.0027, 0.0018])},
 {'IMTA': ([79.1333,
    61.1807,
    42.5867,
    30.0993,
    15.1393,
    10.4193,
    5.4133,
    1.058],
   [0.0449, 0.0335, 0.1919, 0.2804, 0.3083, 0.3413, 0.2103, 0.0892]),
  'IMTA-PA': ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
  'IMTA-CA': ([4.686, 3.9747, 2.554, 1.5093, 0.5273, 0.284, 0.116, 0.0153],
   [0.313, 0.2679, 0.1615, 0.0798, 0.0511, 0.0193, 0.0049, 0.0062])},
 {'IMTA': ([82.066, 60.61, 40.5113, 28.5447, 14.9687, 10.7493, 6.244, 1.7033],
   [0.1294, 0.145, 0.3085, 0.3908, 0.3494, 0.4029, 0.3147, 0.1176]),
  'IMTA-PA': ([1.7607, 0.0007, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
   [0.0105, 0.0009, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
  'IMTA-

# 1 Anytime performance

In [8]:
L = len(logits)
N = logits.shape[1]

In [9]:
probs = torch.softmax(logits, dim=2)
preds = {i: torch.argmax(probs, dim=2)[i, :] for i in range(L)}
acc = [(targets == preds[i]).sum() / len(targets) for i in range(L)]

In [10]:
[round(float(x), 4) for x in acc]

[0.5716, 0.6585, 0.6998, 0.7186, 0.7275]

In [11]:
[round(x, 4) for x in modal_probs_decreasing(preds, probs, L, N=N, thresholds=[-0.0001, -0.01, -0.05, -0.1, -0.2, -0.25, -0.33, -0.5], diffs_type="all").values()]

[79.162, 61.17, 42.818, 30.452, 15.556, 10.884, 5.71, 1.184]

In [13]:
T = 1.
probs_poe_ovr_break_ties_generalized = torch.tensor(f_probs_ovr_poe_logits_weighted_generalized(logits, weights=(np.arange(1, L + 1, 1, dtype=float) / L) * T))
preds_poe_ovr_break_ties_generalized = {i: torch.argmax(probs_poe_ovr_break_ties_generalized, dim=2)[i, :] for i in range(L)}
acc_poe_ovr_break_ties_generalized = [(targets == preds_poe_ovr_break_ties_generalized[i]).sum() / len(targets) for i in range(L)]

In [14]:
[round(float(x), 4) for x in acc_poe_ovr_break_ties_generalized]

[0.5716, 0.6541, 0.6931, 0.7142, 0.7304]

In [15]:
[round(x, 4) for x in modal_probs_decreasing(preds_poe_ovr_break_ties_generalized, 
                                            probs_poe_ovr_break_ties_generalized, L, N=N, thresholds=[-0.0001, -0.01, -0.05, -0.1, -0.2, -0.25, -0.33, -0.5], diffs_type="all").values()]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [16]:
probs_stateful = probs_stateful = anytime_caching(probs, N=N, L=L)
preds_stateful = {i: torch.argmax(probs_stateful, dim=2)[i, :] for i in range(L)}
acc_stateful = [(targets == preds_stateful[i]).sum() / len(targets) for i in range(L)]

In [17]:
[round(float(x), 4) for x in acc_stateful]

[0.5716, 0.6467, 0.6868, 0.7078, 0.7219]

In [18]:
[round(x, 4) for x in modal_probs_decreasing(preds_stateful, probs_stateful, L, N=N, thresholds=[-0.0001, -0.01, -0.05, -0.1, -0.2, -0.25, -0.33, -0.5], diffs_type="all").values()]

[4.704, 4.006, 2.592, 1.588, 0.594, 0.31, 0.122, 0.01]